In [ ]:
import pandas as pd

train_df = pd.read_csv("esun_train.csv",encoding="utf-8")
test_df = pd.read_csv("test.csv",encoding="utf-8")
train_df

In [ ]:
train_df = train_df.fillna("N")

In [ ]:
# column=list(train_df.columns)
# column.remove("fraud_ind")
# print(column)

In [ ]:
train_df["ecfg"] = train_df["ecfg"].apply(lambda x: 1 if x =='Y' else 0)
train_df["flbmk"] = train_df["flbmk"].apply(lambda x: 1 if x =='Y' else 0)
train_df["flg_3dsmk"] = train_df["flg_3dsmk"].apply(lambda x: 1 if x =='Y' else 0)
train_df["insfg"] = train_df["insfg"].apply(lambda x: 1 if x =='Y' else 0)
train_df["ovrlt"] = train_df["ovrlt"].apply(lambda x: 1 if x =='Y' else 0)

In [ ]:
project=train_df
column=["acqic","contp","etymd","mchno","mcc","iterm",
        "scity","stocn","hcefg","csmcu","stscd","locdt","loctm",
        "conam"]
for col in column:
    '''col是欄位，可以修改'''
#     col="csmcu"
    col_count=train_df[col].value_counts()
    bad=[]
    # 取出每個類別值，算出盜刷率
    for i in col_count.index:
        find=train_df[train_df[col]==i]
        true=find[find.fraud_ind == 1]
        present=len(true)/len(find)
        bad.append(present)
    cat="category_"+col
    bad_per="bad_percent_"+col
    test = pd.DataFrame(columns=[cat, bad_per])
    test[cat]=col_count.index
    test[bad_per]=bad
    # test.acount=col_count.values
    '''  >0.03 取盜刷率d大於百分之三的類別值，可以修改'''
    #test[test["bad_percent"]>0.03]
    #test.sort_values(by=['bad_percent'],ascending= False)

    #train_df["csmcu_bad"]=bad
    project=pd.merge(project,test, how='left',
                     left_on=col,right_on=cat)

    project.drop(columns=[cat])
    print(cat)

In [ ]:
project = project.drop(columns=['category_acqic', 'category_contp',
       'category_etymd','category_mchno',  'category_mcc',
       'category_iterm','category_scity',  'category_stocn',
        'category_hcefg','category_csmcu',  'category_stscd',
        'category_locdt','category_loctm',  'category_conam',])
project.columns

In [ ]:
train_df = train_df.drop(columns=["acqic","contp","etymd","mchno","mcc","iterm",
        "scity","stocn","hcefg","csmcu","stscd","locdt","loctm",
        "conam","bacno","cano","txkey"])
train_df.columns()


In [ ]:
train_df.columns

In [ ]:
project.to_csv('esun_train.csv', encoding='utf-8', index=False)

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import statistics as st
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np
import warnings
from sklearn.model_selection import train_test_split
import xgboost as xgb
import re
from sklearn.model_selection import GridSearchCV
import warnings
from sklearn.metrics import accuracy_score
warnings.filterwarnings("ignore")

In [5]:
data = pd.read_table("esun_train_v1.csv", sep=",", encoding="utf-8")
print(data.columns)

Index(['ecfg', 'flbmk', 'flg_3dsmk', 'fraud_ind', 'insfg', 'ovrlt',
       'bad_percent_acqic', 'bad_percent_contp', 'bad_percent_etymd',
       'bad_percent_mchno', 'bad_percent_mcc', 'bad_percent_iterm',
       'bad_percent_scity', 'bad_percent_stocn', 'bad_percent_hcefg',
       'bad_percent_csmcu', 'bad_percent_stscd', 'bad_percent_loctm',
       'bad_percent_conam'],
      dtype='object')


In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop(["fraud_ind"],axis=1)
                                                    , data["fraud_ind"], test_size=0.1, random_state=42)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_train_scale = scaler.fit_transform(X_train)
# x_test_scale = scaler.transform(test)

In [ ]:
# X_train = np.matrix(X_train)
# X_test = np.matrix(X_test)
#
# MR_params = {'n_estimators': [150,160,170,180]}
# MR_params = {'gamma': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6]}
# MR_params = {'subsample': [0.6, 0.7, 0.8, 0.9], 'colsample_bytree': [0.6, 0.7, 0.8, 0.9]}
# MR_params = {'reg_alpha': [0.05, 0.1, 1, 2, 3], 'reg_lambda': [0.05, 0.1, 1, 2, 3]}
MR_params = {'learning_rate': [0.01, 0.05, 0.1, 0.2]}
other_params = {'learning_rate' :1, 'n_estimators':100, 'max_depth':5,
                'min_child_weight':1, 'gamma':0, 'subsample':0.8, 'colsample_bytree':0.8, 
                'objective':"binary:logitraw", 'nthread':4, 'scale_pos_weight':1, 'seed':27}
gbm = xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=gbm, param_grid=MR_params, 
                             scoring='f1', cv=5, verbose=1, n_jobs=-1)
optimized_GBM.fit(X_train, y_train)
evalute_result = optimized_GBM.cv_results_
# print('每輪迭代執行結果:{0}'.format(evalute_result))
print('引數的最佳取值：{0}'.format(optimized_GBM.best_params_))
# print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))

In [7]:
gbm = xgb.XGBClassifier(learning_rate =0.1, n_estimators=100, max_depth=5,
                min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, 
                objective="binary:logitraw", nthread=4, scale_pos_weight=1, seed=27).fit(X_train, y_train)
#比較一下調完跟預設值的差異
# gbm = xgb.XGBRegressor(objective="reg:linear", n_estimators=50, learning_rate=0.2).fit(X_train, y_train)
predictions = gbm.predict(X_test)

In [8]:
predictions

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [9]:
from sklearn.metrics import confusion_matrix


C=confusion_matrix(y_test, predictions)

In [10]:
C

array([[111129,     93],
       [   442,   1028]], dtype=int64)

In [12]:
data_pre = pd.read_table("esun_test_v1.csv", sep=",", encoding="utf-8")
submit_01 = gbm.predict(data_pre)

In [22]:
pd.Series(submit_01).value_counts()

0    420989
1       676
dtype: int64

In [26]:
test_ori = pd.read_csv("test.csv")
test_sub = pd.DataFrame()
test_sub["txkey"]=test_ori.txkey
test_sub["fraud_ind"]=pd.Series(submit_01)
#submit_01.to_csv("result.csv",encoding = "utf-8")


,txkey,fraud_ind
0,592489,0
1,592452,0
2,590212,0
3,590209,0
4,592488,0
5,592460,0
6,590160,0
7,592453,0
8,585745,0
9,592451,0


In [27]:
test_sub.to_csv("result.csv",encoding = "utf-8")

In [ ]:
# '''col是欄位，可以修改'''
# col="csmcu"
# col_count=train_df[col].value_counts()
# '''取前500筆，可以修改'''
# head=100
# bad=[]
# # 取出每個類別值，算出盜刷率
# for i in col_count.index[0:head]:
#     find=train_df[train_df[col]==i]
#     true=find[find.fraud_ind == 1]
#     present=len(true)/len(find)
#     bad.append(present)

# test = pd.DataFrame(columns=["category", "bad_percent","acount"])
# test.category=col_count.index[0:head]
# test.bad_percent=bad
# test.acount=col_count.values[0:head]
# '''  >0.03 取盜刷率d大於百分之三的類別值，可以修改'''
# test[test["bad_percent"]>0.03]
# test.sort_values(by=['bad_percent'])

In [ ]:
# heatmap API: https://seaborn.pydata.org/generated/seaborn.heatmap.html
# colormap: https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html
# import matplotlib.pyplot as plt
# %matplotlib inline
# import seaborn as sns
# plt.figure(figsize=(14, 14))
# sns.heatmap(train_df.corr(), annot=True, cmap="RdBu_r")

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# clf = DecisionTreeClassifier(max_depth=11, min_samples_leaf=20)
# clf = clf.fit(X_train,y_train)

In [ ]:
# from sklearn.tree import export_graphviz
# import graphviz
# data = export_graphviz(clf, out_file = None,
#                       feature_names = X_train.drop(["fraud_ind"],axis = 1).columns,
#                       class_name = ["Y","N"],
#                       filled = True, rounded = True,
#                       special_characters = True)
# graph = graphviz.Source(data)
# graph

In [ ]:
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# clf = RandomForestClassifier()
# grid = {
#     "n_estimators":range(20, 30, 1),
#     "max_depth":range(8, 12)
# }
# gridsearch = GridSearchCV(clf, param_grid=grid, cv=10, n_jobs=8)
# gridsearch.fit(X_train, y_train)
# gridsearch.best_params_

In [ ]:
# ## from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# import numpy as np
# clf = RandomForestClassifier(n_estimators=25, max_depth=11)
# scores = cross_val_score(clf, X_train, y_train, cv=10)
# print("十次:", scores)
# print("平均:", np.average(scores))

In [ ]:
# clf = RandomForestClassifier(n_estimators=25, max_depth=11)
# clf.fit(X_train, y_train)

In [ ]:
# pre = clf.predict(X_test)

In [ ]:
# from sklearn.metrics import classification_report 
# print(classification_report(y_test, pre))